In [1]:
# TẢI DỮ LIỆU ĐƯỢC LƯU TỪ LINK DRIVE (LINK CHỨA DATA)
!pip install -q gdown
import gdown
# File 1
id1 = '1oxWr6TQApjbUA7cL8riCIIIWa4AaN0S2'
url1 = f'https://drive.google.com/uc?id={id1}'
output1 = 'offerings.csv'
gdown.download(url1, output1, quiet=False)

# File 2
id2 = '1NHKihgfjZ_Yy-XdQeRtfgIZ-PBHUAUWW'
url2 = f'https://drive.google.com/uc?id={id2}'
output2 = 'reviews.csv'
gdown.download(url2, output2, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1oxWr6TQApjbUA7cL8riCIIIWa4AaN0S2
To: /content/offerings.csv
100%|██████████| 1.22M/1.22M [00:00<00:00, 95.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1NHKihgfjZ_Yy-XdQeRtfgIZ-PBHUAUWW
From (redirected): https://drive.google.com/uc?id=1NHKihgfjZ_Yy-XdQeRtfgIZ-PBHUAUWW&confirm=t&uuid=395fc667-8b87-4802-b973-aba79ddb995a
To: /content/reviews.csv
100%|██████████| 133M/133M [00:02<00:00, 60.1MB/s]


'reviews.csv'

In [23]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import json
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, r2_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from xgboost import XGBClassifier
import nltk
import scipy.sparse as sp
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')

# Tải tài nguyên NLTK
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

2.1. TIỀN XỬ LÍ DỮ LIỆU

In [3]:
# Tạo thư mục img nếu chưa tồn tại
if not os.path.exists('img'):
    os.makedirs('img')

In [4]:
# Đọc cả 2 file
hotels = pd.read_csv(output1, on_bad_lines='skip')
reviews = pd.read_csv(output2, on_bad_lines='skip')
print(hotels.info())
print(reviews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4333 entries, 0 to 4332
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   hotel_class  3141 non-null   float64
 1   region_id    4333 non-null   int64  
 2   url          4333 non-null   object 
 3   phone        0 non-null      float64
 4   details      0 non-null      float64
 5   address      4333 non-null   object 
 6   type         4333 non-null   object 
 7   id           4333 non-null   int64  
 8   name         4333 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 304.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112900 entries, 0 to 112899
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ratings            112900 non-null  object 
 1   title              112900 non-null  object 
 2   text               112900 non-null  object 
 3   au

In [5]:
# Đổi tên cột và ghép dữ liệu
hotels = hotels.rename(columns={'id': 'offering_id'})
df = pd.merge(reviews, hotels, on='offering_id', how='inner')

# Loại bỏ cột không cần thiết
df = df.drop(['author', 'url', 'phone', 'details', 'id', 'via_mobile'], axis=1)

# Kiểm tra trùng lặp
print("Duplicate rows in hotels:", hotels.duplicated().sum())

# Thêm giá tiền mỗi đêm (giả lập dựa trên hotel_class)
def generate_price(hotel_class):
    if np.isnan(hotel_class):
        return np.nan
    if hotel_class <= 1:
        return np.random.uniform(50, 100)
    elif hotel_class <= 2:
        return np.random.uniform(100, 150)
    elif hotel_class <= 3:
        return np.random.uniform(150, 250)
    elif hotel_class <= 4:
        return np.random.uniform(250, 400)
    else:
        return np.random.uniform(400, 1000)

df['price_per_night'] = df['hotel_class'].apply(generate_price)

Duplicate rows in hotels: 0


In [6]:
# Chuyển đổi cột date và date_stayed sang định dạng datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date_stayed'] = pd.to_datetime(df['date_stayed'], errors='coerce')

In [7]:
import json
# Hàm xử lý chuỗi JSON
def clean_json(x):
    if isinstance(x, str):
        return json.loads(x.replace("'", '"'))
    elif isinstance(x, dict):
        return x
    else:
        raise ValueError(f"Unexpected type for ratings: {type(x)}")

# Xử lý cột ratings
df['ratings'] = df['ratings'].apply(clean_json)
normalized_cols = pd.json_normalize(df['ratings'])

# Xóa các cột trùng lặp trong df
duplicate_columns = [col for col in normalized_cols.columns if col in df.columns]
df = df.drop(columns=duplicate_columns, errors='ignore')

# Ghép normalized_cols vào df
df = df.join(normalized_cols)

# Xóa cột ratings
df = df.drop(['ratings'], axis=1)

# Xử lý giá trị thiếu cho các cột hiện có
df['date_stayed'] = df['date_stayed'].fillna('Unknown')
df['hotel_class'] = df['hotel_class'].fillna(df['hotel_class'].median())
df['price_per_night'] = df['price_per_night'].fillna(df['price_per_night'].median())
df['text'] = df['text'].fillna('No content')

# Xử lý giá trị thiếu cho các cột đánh giá
rating_columns = [
    'service', 'cleanliness', 'value', 'rooms', 'check_in_front_desk',
    'location', 'sleep_quality', 'business_service_(e_g_internet_access)'
]

# Điền giá trị thiếu bằng trung vị theo hotel_class
for col in rating_columns:
    df[col] = df.groupby('hotel_class')[col].transform(lambda x: x.fillna(x.median()))

# Điền giá trị thiếu còn lại bằng trung vị tổng thể
for col in rating_columns:
    df[col] = df[col].fillna(df[col].median())

# Kiểm tra lại giá trị thiếu
print("Giá trị thiếu sau khi xử lý:")
print(df.isnull().sum())

# Kiểm tra kết quả
print(df.head())

Giá trị thiếu sau khi xử lý:
title                                     0
text                                      0
date_stayed                               0
offering_id                               0
num_helpful_votes                         0
date                                      0
sentiment                                 0
hotel_class                               0
region_id                                 0
address                                   0
type                                      0
name                                      0
price_per_night                           0
overall                                   0
service                                   0
cleanliness                               0
value                                     0
rooms                                     0
check_in_front_desk                       0
location                                  0
sleep_quality                             0
business_service_(e_g_internet_access)    0
dty

In [8]:
# Phân tích cảm xúc từ văn bản bằng VADER
sid = SentimentIntensityAnalyzer()

def get_sentiment(text):
    scores = sid.polarity_scores(text)
    compound = scores['compound']
    if compound >= 0.05:
        return 'Positive'
    elif compound <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

df['sentiment'] = df['text'].apply(get_sentiment)

In [9]:
# Chuẩn hóa đặc trưng số
scaler = StandardScaler()
numeric_cols = ['num_helpful_votes', 'hotel_class', 'price_per_night']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [10]:
# Xử lý giá trị ngoại lệ bằng IQR
for col in ['num_helpful_votes', 'price_per_night']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df[col] = df[col].clip(lower=Q1 - 1.5*IQR, upper=Q3 + 1.5*IQR)

2.2. PHÂN TÍCH THỐNG KÊ SƠ BỘ

In [11]:
# Phân tích thống kê
print("Thống kê mô tả cho các đặc trưng số:")
print(df[numeric_cols].describe())

# Thống kê tổng thể
print("\nPhân phối nhãn sentiment:")
print(df['sentiment'].value_counts(normalize=True))

# Thống kê TF-IDF (tính sau khi xử lý văn bản)
tfidf = TfidfVectorizer(max_features=1000, stop_words='english', ngram_range=(1, 2))
tfidf_matrix = tfidf.fit_transform(df['text'])
print("\nKích thước ma trận TF-IDF:", tfidf_matrix.shape)
print("Số lượng giá trị khác 0 trong TF-IDF:", tfidf_matrix.nnz)
print("Tỷ lệ thưa thớt:", tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1]))

Thống kê mô tả cho các đặc trưng số:
       num_helpful_votes   hotel_class  price_per_night
count      112899.000000  1.128990e+05    112899.000000
mean           -0.178952 -6.444661e-17        -0.099457
std             0.343000  1.000004e+00         0.706869
min            -0.462257 -3.338920e+00        -1.630344
25%            -0.462257 -6.802791e-01        -0.608539
50%            -0.462257 -1.561877e-02        -0.149396
75%            -0.104027  6.490415e-01         0.266496
max             0.433318  1.978362e+00         1.579049

Phân phối nhãn sentiment:
sentiment
Positive    0.732805
Negative    0.187734
Neutral     0.079460
Name: proportion, dtype: float64

Kích thước ma trận TF-IDF: (112899, 1000)
Số lượng giá trị khác 0 trong TF-IDF: 4482701
Tỷ lệ thưa thớt: 0.03970540925960372


In [12]:
# 1. Histogram cho phân phối của điểm đánh giá tổng thể và giá tiền mỗi đêm
plt.figure(figsize=(10, 5))
sns.histplot(df['overall'], bins=5, kde=True)
plt.title('Phân phối của điểm đánh giá tổng thể (overall)')
plt.xlabel('Điểm đánh giá')
plt.ylabel('Số lượng')
plt.savefig('img/average_rating_distribution_hotel.png')
plt.close()

plt.figure(figsize=(10, 5))
sns.histplot(df['price_per_night'], bins=30, kde=True)
plt.title('Phân phối của giá tiền mỗi đêm (price_per_night)')
plt.xlabel('Giá tiền ($)')
plt.ylabel('Số lượng')
plt.savefig('img/price_distribution_hotel.png')
plt.close()

In [13]:
# 2. Ma trận tương quan giữa các cột đánh giá và giá tiền
corr_columns = ['overall', 'service', 'rooms', 'cleanliness', 'sleep_quality', 'location', 'value', 'price_per_night']
corr_matrix = df[corr_columns].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Ma trận tương quan giữa các cột đánh giá và giá tiền')
plt.savefig('img/correlation_matrix_hotel.png')
plt.close()

In [14]:
# 3. Biểu đồ đường cho xu hướng điểm đánh giá theo thời gian
df['month'] = df['date'].dt.to_period('M')
monthly_avg = df.groupby('month')['overall'].mean().reset_index()
monthly_avg['month'] = monthly_avg['month'].dt.to_timestamp()

In [15]:
# 4. Biểu đồ cột cho phân phối cảm xúc
sentiment_counts = df['sentiment'].value_counts()

plt.figure(figsize=(8, 6))
sentiment_counts.plot(kind='bar', color=['green', 'yellow', 'red'])
plt.title('Phân phối của cảm xúc trong đánh giá')
plt.xlabel('Cảm xúc')
plt.ylabel('Số lượng')
plt.savefig('img/sentiment_distribution.png')
plt.close()

In [16]:
# 5. Boxplot cho các giá trị ngoại lệ
plt.figure(figsize=(10, 5))
sns.boxplot(x=df['num_helpful_votes'])
plt.title('Boxplot cho số lượng phiếu hữu ích (num_helpful_votes)')
plt.savefig('img/helpful_votes_boxplot.png')
plt.close()

plt.figure(figsize=(10, 5))
sns.boxplot(x=df['price_per_night'])
plt.title('Boxplot cho giá tiền mỗi đêm (price_per_night)')
plt.savefig('img/price_boxplot.png')
plt.close()

2.4. SỬ DỤNG CÁC MÔ HÌNH PHÂN LOẠI VÀ GIẢM CHIỀU

In [17]:
df_sample = df.sample(n=20000, random_state=42)
X = df_sample[numeric_cols + ['text']]
y = df_sample['sentiment']

# Chia dữ liệu thành train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Xử lý TF-IDF
tfidf_train = tfidf.fit_transform(X_train['text'])
tfidf_test = tfidf.transform(X_test['text'])

# Kết hợp đặc trưng số và TF-IDF
X_train_combined = np.hstack([X_train[numeric_cols].to_numpy(), tfidf_train.toarray()])
X_test_combined = np.hstack([X_test[numeric_cols].to_numpy(), tfidf_test.toarray()])

# Giảm chiều với PCA
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_combined)
X_test_pca = pca.transform(X_test_combined)
print("\nSố chiều sau PCA:", X_train_pca.shape[1])
print("Phương sai giải thích (PCA):", pca.explained_variance_ratio_)
print("Tổng phương sai bảo tồn (PCA):", sum(pca.explained_variance_ratio_))

# Giảm chiều với TruncatedSVD
svd = TruncatedSVD(n_components=100, random_state=42)
X_train_svd = svd.fit_transform(sp.hstack([sp.csr_matrix(X_train[numeric_cols]), tfidf_train]))
X_test_svd = svd.transform(sp.hstack([sp.csr_matrix(X_test[numeric_cols]), tfidf_test]))
print("\nSố chiều sau TruncatedSVD:", X_train_svd.shape[1])
print("Phương sai giải thích (SVD):", svd.explained_variance_ratio_)
print("Tổng phương sai bảo tồn (SVD):", sum(svd.explained_variance_ratio_))


Số chiều sau PCA: 631
Phương sai giải thích (PCA): [5.53786818e-01 4.77283025e-02 3.82126182e-02 1.14994322e-02
 8.35859330e-03 6.47352429e-03 5.31427017e-03 3.67690804e-03
 2.97052561e-03 2.74532885e-03 2.49713057e-03 2.16851290e-03
 2.12685895e-03 1.96943040e-03 1.89377697e-03 1.79546127e-03
 1.77042725e-03 1.71780468e-03 1.63458712e-03 1.60899725e-03
 1.53460432e-03 1.46859248e-03 1.42842155e-03 1.35647414e-03
 1.28341009e-03 1.27765711e-03 1.25989254e-03 1.23867066e-03
 1.19972997e-03 1.17822064e-03 1.16525094e-03 1.14335437e-03
 1.10011208e-03 1.08758203e-03 1.07284130e-03 1.06747323e-03
 1.03735625e-03 1.03426328e-03 1.02408090e-03 1.01993919e-03
 9.86382167e-04 9.73892761e-04 9.70569439e-04 9.48953479e-04
 9.44007791e-04 9.38979622e-04 9.30657091e-04 9.20990344e-04
 9.04014086e-04 8.97491523e-04 8.89360573e-04 8.81316351e-04
 8.68380788e-04 8.60988787e-04 8.49803259e-04 8.47352726e-04
 8.31287720e-04 8.28713100e-04 8.18628048e-04 8.02206245e-04
 7.98097277e-04 7.94430736e-04 7.

In [18]:
# Chuẩn hóa dữ liệu giảm chiều
scaler_pca = StandardScaler()
X_train_pca_scaled = scaler_pca.fit_transform(X_train_pca)
X_test_pca_scaled = scaler_pca.transform(X_test_pca)

scaler_svd = StandardScaler()
X_train_svd_scaled = scaler_svd.fit_transform(X_train_svd)
X_test_svd_scaled = scaler_svd.transform(X_test_svd)

# Thống kê mô tả cho dữ liệu sau giảm chiều
print("\nThống kê mô tả cho dữ liệu sau PCA:")
print(pd.DataFrame(X_train_pca_scaled).describe())

print("\nThống kê mô tả cho dữ liệu sau TruncatedSVD:")
print(pd.DataFrame(X_train_svd_scaled).describe())


Thống kê mô tả cho dữ liệu sau PCA:
                0             1             2             3             4    \
count  1.600000e+04  1.600000e+04  1.600000e+04  1.600000e+04  1.600000e+04   
mean   3.552714e-18 -4.884981e-18 -1.021405e-17 -1.065814e-17 -7.105427e-18   
std    1.000031e+00  1.000031e+00  1.000031e+00  1.000031e+00  1.000031e+00   
min   -3.044827e+00 -1.279653e+00 -2.242985e+00 -1.710693e+00 -2.035853e+00   
25%   -7.484187e-01 -8.260452e-01 -7.129248e-01 -6.666044e-01 -2.767152e-01   
50%    2.008796e-01 -5.236492e-01 -6.376537e-02 -3.020870e-01 -1.422530e-01   
75%    5.804145e-01  6.163085e-01  6.050320e-01  2.818041e-01  2.323229e-02   
max    2.167092e+00  2.166464e+00  2.834406e+00  4.301387e+00  6.006465e+00   

                5             6             7             8             9    \
count  1.600000e+04  1.600000e+04  1.600000e+04  1.600000e+04  1.600000e+04   
mean  -2.664535e-18 -4.884981e-18 -1.509903e-17  7.549517e-18  2.020606e-17   
std    1.00003

In [19]:
# Trực quan hóa 4 cặp thành phần chính (PCA)
plt.figure(figsize=(12, 10))
for i in range(4):
    plt.subplot(2, 2, i+1)
    sns.scatterplot(x=X_train_pca_scaled[:, i], y=X_train_pca_scaled[:, i+1], hue=y_train, palette='viridis', alpha=0.6)
    plt.title(f'PCA: PC{i+1} vs PC{i+2}')
    plt.xlabel(f'PC{i+1}')
    plt.ylabel(f'PC{i+2}')
plt.tight_layout()
plt.savefig('img/pca_pairs.png')
plt.close()

# Trực quan hóa 4 cặp thành phần chính (SVD)
plt.figure(figsize=(12, 10))
for i in range(4):
    plt.subplot(2, 2, i+1)
    sns.scatterplot(x=X_train_svd_scaled[:, i], y=X_train_svd_scaled[:, i+1], hue=y_train, palette='viridis', alpha=0.6)
    plt.title(f'SVD: Component {i+1} vs Component {i+2}')
    plt.xlabel(f'Component {i+1}')
    plt.ylabel(f'Component {i+2}')
plt.tight_layout()
plt.savefig('img/svd_pairs.png')
plt.close()

In [20]:
# Tạo DataFrame từ X_train_pca_scaled và y_train để trực quan hóa
pca_df = pd.DataFrame(X_train_pca_scaled, columns=[f'PC{i+1}' for i in range(X_train_pca_scaled.shape[1])])
pca_df['sentiment'] = y_train.reset_index(drop=True)

# Trực quan hóa mối quan hệ giữa thành phần chính và sentiment (PCA)
plt.figure(figsize=(12, 4))
for i in range(3):
    plt.subplot(1, 3, i+1)
    sns.boxplot(x='sentiment', y=f'PC{i+1}', data=pca_df, order=['Positive', 'Neutral', 'Negative'])
    plt.title(f'PCA: PC{i+1} vs Sentiment')
    plt.xlabel('Sentiment')
    plt.ylabel(f'PC{i+1}')
plt.tight_layout()
plt.savefig('img/pca_sentiment_boxplot.png')
plt.close()

# Tạo DataFrame từ X_train_svd_scaled và y_train để trực quan hóa
svd_df = pd.DataFrame(X_train_svd_scaled, columns=[f'Component {i+1}' for i in range(X_train_svd_scaled.shape[1])])
svd_df['sentiment'] = y_train.reset_index(drop=True)

# Trực quan hóa mối quan hệ giữa thành phần chính và sentiment (SVD)
plt.figure(figsize=(12, 4))
for i in range(3):
    plt.subplot(1, 3, i+1)
    sns.boxplot(x='sentiment', y=f'Component {i+1}', data=svd_df, order=['Positive', 'Neutral', 'Negative'])
    plt.title(f'SVD: Component {i+1} vs Sentiment')
    plt.xlabel('Sentiment')
    plt.ylabel(f'Component {i+1}')
plt.tight_layout()
plt.savefig('img/svd_sentiment_boxplot.png')
plt.close()

In [21]:
# So sánh PCA và TruncatedSVD
print("\nSo sánh PCA và TruncatedSVD:")
print(f"PCA: {X_train_pca.shape[1]} chiều, bảo tồn {sum(pca.explained_variance_ratio_):.2%} phương sai")
print(f"SVD: {X_train_svd.shape[1]} chiều, bảo tồn {sum(svd.explained_variance_ratio_):.2%} phương sai")

# Đánh giá thời gian xử lý
start_time = time.time()
pca.fit(X_train_combined)
pca_time = time.time() - start_time

start_time = time.time()
svd.fit(sp.hstack([sp.csr_matrix(X_train[numeric_cols]), tfidf_train]))
svd_time = time.time() - start_time

print(f"Thời gian PCA: {pca_time:.2f} giây")
print(f"Thời gian SVD: {svd_time:.2f} giây")


So sánh PCA và TruncatedSVD:
PCA: 631 chiều, bảo tồn 95.00% phương sai
SVD: 100 chiều, bảo tồn 76.92% phương sai
Thời gian PCA: 5.95 giây
Thời gian SVD: 1.00 giây


In [24]:
# Định nghĩa các mô hình phân loại
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, C=1.0, class_weight='balanced'),
    "SVM": SVC(kernel='linear', C=1.0, class_weight='balanced', probability=True),
    "XGBoost": XGBClassifier(
        max_depth=7,
        learning_rate=0.001,
        n_estimators=300,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=1  # Có thể điều chỉnh nếu cần xử lý mất cân bằng
    )
}


# Tỷ lệ train:validation
ratios = [(0.8, 0.2), (0.7, 0.3), (0.6, 0.4)]
results = []

In [25]:
# Kiểm tra kích thước dữ liệu
print("Kích thước X_train_pca:", X_train_pca.shape)
print("Kích thước X_test_pca:", X_test_pca.shape)
print("Kích thước X_train_svd:", X_train_svd.shape)
print("Kích thước X_test_svd:", X_test_svd.shape)
print("Kích thước y_train:", y_train.shape)
print("Kích thước y_test:", y_test.shape)
# Đặt lại chỉ số của y_train và y_test để khớp với X_train_pca và X_test_pca
if isinstance(y_train, pd.Series):
    y_train = y_train.reset_index(drop=True)
if isinstance(y_test, pd.Series):
    y_test = y_test.reset_index(drop=True)

# Kiểm tra lại kích thước sau khi đặt lại chỉ số
print("\nSau khi đặt lại chỉ số:")
print("Kích thước y_train:", y_train.shape)
print("Kích thước y_test:", y_test.shape)

Kích thước X_train_pca: (16000, 631)
Kích thước X_test_pca: (4000, 631)
Kích thước X_train_svd: (16000, 100)
Kích thước X_test_svd: (4000, 100)
Kích thước y_train: (16000,)
Kích thước y_test: (4000,)

Sau khi đặt lại chỉ số:
Kích thước y_train: (16000,)
Kích thước y_test: (4000,)


In [26]:
# Hàm đánh giá mô hình
def evaluate_model(model, X_train, X_test, y_train, y_test, name, ratio, data_type):
    start_time = time.time()
    print(f"Evaluating {name} ({data_type}, Ratio {ratio})...")
    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Đánh giá trên tập train để kiểm tra overfitting
        y_pred_train = model.predict(X_train)
        train_acc = accuracy_score(y_train, y_pred_train)
        test_acc = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True)

        result = {
            'Model': name,
            'Ratio': ratio,
            'Data': data_type,
            'Train Accuracy': train_acc,
            'Test Accuracy': test_acc,
            'Precision': report['weighted avg']['precision'],
            'Recall': report['weighted avg']['recall'],
            'F1': report['weighted avg']['f1-score']
        }
        results.append(result)
        print(f"Result for {name}: {result}")

        # Confusion Matrix
        cm = confusion_matrix(y_test, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
        disp.plot(cmap='Blues')
        plt.title(f"Confusion Matrix - {name} ({data_type}, Ratio {ratio})")
        plt.savefig(f'img/cm_{name.lower().replace(" ", "_")}_{data_type}_{ratio}.png')
        plt.close()

        print(f"{name} ({data_type}, Ratio {ratio}) completed in {time.time() - start_time:.2f} seconds.")
        return y_pred
    except Exception as e:
        print(f"Error evaluating {name} ({data_type}, Ratio {ratio}): {str(e)}")
        return None

In [27]:
# (a) Phân loại với các tỷ lệ train:validation khác nhau
start_time = time.time()
print("Starting classification...")

# Vì đã có X_train_pca, X_test_pca, X_train_svd, X_test_svd từ bước trước
# Chỉ cần đánh giá với các tỷ lệ train:test khác nhau
ratios = [(0.8, 0.2), (0.7, 0.3), (0.6, 0.4)]

for train_ratio, test_ratio in ratios:
    print(f"\nProcessing ratio {train_ratio}:{test_ratio}...")

    # Đánh giá trên dữ liệu giảm chiều bằng PCA
    # Lấy mẫu lại từ X_train_pca, X_test_pca để tạo các tỷ lệ khác nhau
    idx_train_pca = int(len(X_train_pca) * train_ratio / 0.8)  # 0.8 là tỷ lệ train ban đầu
    idx_test_pca = int(len(X_test_pca) * test_ratio / 0.2)     # 0.2 là tỷ lệ test ban đầu
    X_train_pca_subset = X_train_pca[:idx_train_pca]
    X_test_pca_subset = X_test_pca[:idx_test_pca]
    y_train_subset = y_train[:idx_train_pca]
    y_test_subset = y_test[:idx_test_pca]

    for name, model in models.items():
        y_pred = evaluate_model(model, X_train_pca_subset, X_test_pca_subset, y_train_subset, y_test_subset, name, f"{train_ratio}:{test_ratio}", "PCA")

    # Đánh giá trên dữ liệu giảm chiều bằng SVD
    idx_train_svd = int(len(X_train_svd) * train_ratio / 0.8)
    idx_test_svd = int(len(X_test_svd) * test_ratio / 0.2)
    X_train_svd_subset = X_train_svd[:idx_train_svd]
    X_test_svd_subset = X_test_svd[:idx_test_svd]

    for name, model in models.items():
        y_pred = evaluate_model(model, X_train_svd_subset, X_test_svd_subset, y_train_subset, y_test_subset, name, f"{train_ratio}:{test_ratio}", "SVD")

Starting classification...

Processing ratio 0.8:0.2...
Evaluating Logistic Regression (PCA, Ratio 0.8:0.2)...
Result for Logistic Regression: {'Model': 'Logistic Regression', 'Ratio': '0.8:0.2', 'Data': 'PCA', 'Train Accuracy': 0.843625, 'Test Accuracy': 0.8365, 'Precision': 0.8700816618862109, 'Recall': 0.8365, 'F1': 0.8466518089716788}
Logistic Regression (PCA, Ratio 0.8:0.2) completed in 22.20 seconds.
Evaluating SVM (PCA, Ratio 0.8:0.2)...
Result for SVM: {'Model': 'SVM', 'Ratio': '0.8:0.2', 'Data': 'PCA', 'Train Accuracy': 0.838375, 'Test Accuracy': 0.82875, 'Precision': 0.8696636761522232, 'Recall': 0.82875, 'F1': 0.8408722867561408}
SVM (PCA, Ratio 0.8:0.2) completed in 287.14 seconds.
Evaluating XGBoost (PCA, Ratio 0.8:0.2)...
Error evaluating XGBoost (PCA, Ratio 0.8:0.2): Invalid classes inferred from unique values of `y`.  Expected: [0 1 2], got ['Negative' 'Neutral' 'Positive']
Evaluating Logistic Regression (SVD, Ratio 0.8:0.2)...
Result for Logistic Regression: {'Model': 

In [28]:
# Hiển thị kết quả
results_df = pd.DataFrame(results)
print("\nClassification Results:")
print(results_df)


Classification Results:
                  Model    Ratio Data  Train Accuracy  Test Accuracy  \
0   Logistic Regression  0.8:0.2  PCA        0.843625        0.83650   
1                   SVM  0.8:0.2  PCA        0.838375        0.82875   
2   Logistic Regression  0.8:0.2  SVD        0.800312        0.81200   
3                   SVM  0.8:0.2  SVD        0.799188        0.80850   
4   Logistic Regression  0.7:0.3  PCA        0.846357        0.83625   
5                   SVM  0.7:0.3  PCA        0.839857        0.82850   
6   Logistic Regression  0.7:0.3  SVD        0.802571        0.81125   
7                   SVM  0.7:0.3  SVD        0.800071        0.80850   
8   Logistic Regression  0.6:0.4  PCA        0.848500        0.83400   
9                   SVM  0.6:0.4  PCA        0.842917        0.82675   
10  Logistic Regression  0.6:0.4  SVD        0.804833        0.81125   
11                  SVM  0.6:0.4  SVD        0.802000        0.80875   

    Precision   Recall        F1  
0  